In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matminer.featurizers.base import MultipleFeaturizer, StackedFeaturizer
from matminer.featurizers import composition as cf
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, ShuffleSplit, LeaveOneGroupOut, cross_val_score, learning_curve, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn.metrics import roc_curve, auc, r2_score, make_scorer
from sklearn import metrics
from pymatgen import Composition
import pickle as pkl
import pandas as pd
import numpy as np
import gzip
import os

In [ ]:
def featCleanImpute(Y):
    ''' Convert inf to NaN in feature array, in place
    
    Args: 
        Y, feature array, list of ndarray (#samples x #features)
    Returns:
        None
    '''
    # Clean inf values
    for i in range(len(Y)):
        for j in range(len(Y[i])):
            if Y[i][j] == np.inf:
                Y[i][j] = np.nan
            else:
                Y[i][j] = Y[i][j]
                
    # Impute nan values
    imp = Imputer(missing_values='NaN', axis=0, strategy='mean', copy=False)
    imp2 = Imputer(missing_values='NaN', axis=1, strategy='mean', copy=False)

    imp.fit(Y)
    imp.transform(Y)
    imp2.fit(Y)
    imp.transform(Y)
    

# Import and Format datasets

In [ ]:
elastProp_SLAC = pd.read_excel('datasets/Mechnical properties analysis v33_Pruned.xlsx')
rawCopy = elastProp_SLAC.copy()
len(rawCopy)

In [ ]:
elastProp_SLAC.head()

In [ ]:
elastProp_SLAC.rename(index=str, columns={'θd':'theta_d', 'ρ (g/cm3)':'density (g/cm3)', 
                                          'Yeild strength, σy (MPa)':'Yeild strength (MPa)', 
                                         'Compressive fracture strength, σf (MPa)': 'Compressive fracture strength (MPa)'},inplace=True)
#elastProp_SLAC.columns

In [ ]:
dfSLAC = elastProp_SLAC

In [ ]:
def makeComp(x):
    '''Apply Composition() constructor to string x
       Return: Composition object, or 0 if not parse-able'''
    try:
        return Composition(x)
    except:
        print(x)
        return 0

In [ ]:
dfSLAC['comp'] = dfSLAC['Compositions'].apply(makeComp)

Remove entries where composition was unable to be read

In [ ]:
dfSLAC = dfSLAC[dfSLAC['comp']!=0]
dfSLAC.reset_index(drop=True, inplace=True)
print('{} entries remaining'.format(len(dfSLAC)))

In [ ]:
cols = dfSLAC.columns.tolist()
dfSLAC = dfSLAC[cols[-1:] + cols[:-1]]
print("moved 'comp' to first column")

In [ ]:
len(dfSLAC['comp']!=0)

In [ ]:
def redComp(x):
    try: 
        return Composition(x.reduced_formula)
    except: 
        print(x)
        return 0
    
dfSLAC['comp'] = dfSLAC['comp'].apply(redComp)
dfSLAC = dfSLAC[dfSLAC['comp']!=0]
dfSLAC.reset_index(drop=True, inplace=True)
print('{} entries remaining'.format(len(dfSLAC)))

In [ ]:
dfSLAC['compStr'] = dfSLAC['comp'].apply(lambda x: x.reduced_formula)

In [ ]:
dfSLAC.head()

Import NREL dataset

In [ ]:
NRELmodulusDataPath= 'C:\\Users\\Hikaru\\Desktop\\School\\_Stanford\\_SLAC\\MechPropModels\\ryan\'s code\\MG_elastic_properties_package\\datasets\\SLAC_Modulus_Only_Cleaned_modified_trim.xlsx'
elastProp_NREL = pd.read_excel(NRELmodulusDataPath)
rawCopy = elastProp_NREL.copy()
rawCopy.head()

In [ ]:
elastProp_NREL['comp'] = elastProp_NREL['Compositions'].apply(makeComp)
elastProp_NREL['comp'] = elastProp_NREL['comp'].apply(redComp)

In [ ]:
elastProp_NREL['comp'][0]
# Compositions have been standardized and reduced

In [ ]:
# cell about Composition.almost_equal() method experimentation was here
# Note that an unreduced chemical formula is not equal in composition

# Build individual datasets

### Generate, clean feature set

In [ ]:
base_featurizer = MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"),
                                 cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True),
                                cf.YangSolidSolution(), cf.AtomicPackingEfficiency()])

### Young's Modulus dataset 
Isolate Youngs Modulus, Composition.
Build Features

In [ ]:
# Separate Data into new frame
youngsSLAC = pd.DataFrame()
youngsSLAC['comp'] = dfSLAC['comp']
youngsSLAC['E'] = dfSLAC['Young’s  Modulus, E (GPa)']

# Drop missing entries 
youngsSLAC = youngsSLAC[[(type(x) in (int, float)) for x in youngsSLAC['E']]]
youngsSLAC.reset_index(drop=True, inplace=True)
youngsSLAC = youngsSLAC[~youngsSLAC['E'].isnull()]
youngsSLAC['E'].astype(float)
youngsSLAC.reset_index(drop=True, inplace=True)

# Separate Data into new frame
youngsNREL = pd.DataFrame()
youngsNREL['comp'] = elastProp_NREL['comp']
youngsNREL['E'] = elastProp_NREL['Youngs_Modulus_GPa']

# Drop missing entries
youngsNREL = youngsNREL[[(type(x) in (int, float)) for x in youngsNREL['E']]]
youngsNREL.reset_index(drop=True, inplace=True)
youngsNREL = youngsNREL[~youngsNREL['E'].isnull()]
youngsNREL['E'].astype(float)
youngsNREL.reset_index(drop=True, inplace=True)

In [ ]:
# Join and sort data (SLAC dataset is complete)
youngsData = youngsSLAC #.append(youngsNREL, ignore_index=True)
youngsData = youngsData.sort_values('comp').reset_index(drop=True)
print('{} = {} + {}?'.format(len(youngsData), len(youngsSLAC), len([])))

In [ ]:
%%time
X_E = base_featurizer.featurize_many(youngsData['comp'], ignore_errors=True)
X_E = np.array(X_E)
X_E.astype(float)
print('Computed {} features'.format(X_E.shape[1]))

In [ ]:
# Create smaller dataset for quick tests
youngsDataSmall = youngsData.sample(500).reset_index(drop=True)

In [ ]:
XEsmall = base_featurizer.featurize_many(youngsDataSmall['comp'], ignore_errors=True)
XEsmall = np.array(XEsmall)
XEsmall.astype(float)
print('Computed {} features'.format(XEsmall.shape[1]))

In [ ]:
# some duplicates exist
comp = Composition('Zr14Al4Co7')
comp2 = Composition('Zr20Al4Co7') 
for index, row in youngsData[ [x.almost_equals(comp) for x in youngsData['comp']] ].iterrows():
    print('Comp: {} .... at row: {}'.format(row['comp'], index))

In [ ]:
print(np.where(np.isnan(X_E)))
featCleanImpute(X_E)
print(np.where(np.isnan(X_E)))

In [ ]:
with gzip.open('./datasets/youngs_features.pkl.gz', 'wb') as fp:
    pkl.dump(X_E, fp)
with gzip.open('./datasets/youngs_data.pkl.gz', 'wb') as fd:
    pkl.dump(youngsData, fd)

In [ ]:
with gzip.open('./datasets/youngsSmall_features.pkl.gz', 'wb') as fp:
    pkl.dump(XEsmall, fp)
with gzip.open('./datasets/youngsSmall_data.pkl.gz', 'wb') as fd:
    pkl.dump(youngsDataSmall, fd)

In [ ]:
youngsDataReduced = pd.DataFrame()
yDcopy = youngsData.copy()
while ~yDcopy.empty:
    compTemp = yDcopy['comp'][0]
    dupes = yDcopy[ [x.almost_equals(comp) for x in yDcopy['comp']] ]
    
    
    
    break
    yDcopy.reset_index(drop=True, inplace=True)

In [ ]:
dupes

In [ ]:
dfSLAC.columns

### Density Model Dataset
Use as benchmark for validity of other models?

In [ ]:
densitySLAC = pd.DataFrame()
densitySLAC['comp'] = dfSLAC['comp']
densitySLAC['density'] = dfSLAC['density (g/cm3)']

densitySLAC.dropna(inplace=True)
densitySLAC.reset_index(drop=True, inplace=True)
densitySLAC = densitySLAC[[(type(x) in (int, float)) for x in densitySLAC['density']]]
densitySLAC['density'].astype(float)
densitySLAC.reset_index(drop=True, inplace=True)

In [ ]:
print('dropping N/A, non number density values leaves {} values'.format(len(densitySLAC)))
densitySLAC.head()

In [ ]:
densityNREL = pd.DataFrame()
densityNREL['comp'] = elastProp_NREL['comp']
densityNREL['density'] = elastProp_NREL['density']

In [ ]:
densityNREL.dropna(inplace=True)
densityNREL.reset_index(inplace=True)
densityNREL = densityNREL[[(type(x) in (int, float)) for x in densityNREL['density']]]
densityNREL.reset_index(drop=True, inplace=True)
densityNREL['density'].astype(float)
densityNREL.head()

In [ ]:
densityData = densitySLAC.append(densityNREL, ignore_index=True)
print('{} = {} + {}?'.format(len(densityData), len(densitySLAC), len(densityNREL)))

In [ ]:
%%time
X_dens = base_featurizer.featurize_many(densityData['comp'], ignore_errors=True)
X_dens = np.array(X_dens)
X_dens.astype(float)
print('Computed {} features'.format(X_dens.shape[1]))

In [ ]:
print(np.where(np.isnan(X_dens)))
featCleanImpute(X_dens)
print(np.where(np.isnan(X_dens)))

In [ ]:
with gzip.open('./datasets/density_features.pkl.gz', 'wb') as fp:
    pkl.dump(X_dens, fp)
with gzip.open('./datasets/density_data.pkl.gz', 'wb') as fd:
    pkl.dump(densityData, fd)

-------------------------

# Build Model: Density

Build model, Impute unknown values

In [ ]:
density_model = Pipeline([('impute',Imputer()), 
                          ('model', RandomForestRegressor(n_estimators=100, n_jobs=1, max_features=12))])

In [ ]:
%%time
density_model.fit(X_dens, densitySLAC['density'])

In [ ]:
featureImp = pd.DataFrame(density_model.steps[1][1].feature_importances_,
                          index=dens_featurizer.feature_labels(),
                          columns=['importance']).sort_values('importance',ascending=False)
featureImp.head()

# Assess quality of model?
what metric to use?  r2 seems fine

In [ ]:
train_sizes, train_scores, valid_scores = learning_curve(density_model, X_dens, densitySLAC['density'], cv=ShuffleSplit())#,
                                                          #scoring=r2_scorer)

In [ ]:
plt.figure()
plt.title('Learning Curve, RForest: std model, 12 features')
plt.xlabel("Training examples: density data")
plt.ylabel("default scoring method?")
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")
plt.show()

In [ ]:
r2_scorer = make_scorer(r2_score)
train_sizes, train_scores, valid_scores = learning_curve(density_model, X_dens, densitySLAC['density'], cv=ShuffleSplit(),
                                                          scoring=r2_scorer)

In [ ]:
plt.figure()
plt.title('Learning Curve, RForest: std features, max 12 features')
plt.xlabel("Training examples: density data")
plt.ylabel("$r^2$")
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(valid_scores, axis=1)
test_scores_std = np.std(valid_scores, axis=1)
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")
plt.show()

Plot predicted vs actual scatter plot

In [ ]:
#take predicted values from each cross validation set to ensure training not performed on own set.  Take 200 training samples
# Probably could have used cross_val_predict
kf = KFold(5)
rep=0
densitySLAC['density_predict'] = np.nan
for train_index, test_index in kf.split(densitySLAC['comp']):
    print('Split #{}'.format(rep))
    density_model.fit(X_dens[train_index,:], densitySLAC['density'][train_index])
    
    y_densPredict = density_model.predict(X_dens[test_index,:])
    densitySLAC['density_predict'][test_index] = y_densPredict
    
    #print(train_index, test_index)
    rep+=1
#y_densPredict = 

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(densitySLAC['density'], densitySLAC['density_predict'], edgecolors='k')
plt.plot([0,30], [0,30], 'r-')
plt.xlabel('density [g/cm$^3$]')
plt.ylabel('predicted density')

plt.xlim([0,25])
plt.ylim([0,25])


# Build Model: Modulus

In [ ]:
elasticSLAC = pd.DataFrame()
elasticSLAC['comp'] = dfSLAC['comp']
elasticSLAC['E'] = dfSLAC['Young’s  Modulus, E (GPa)']

In [ ]:
elastic_model = Pipeline([('impute',Imputer()), ('model', RandomForestRegressor(n_estimators=100, n_jobs=1, max_features=12))])